In [1]:
import pandas as pd
import requests
import firebase_admin
from firebase_admin import credentials, storage

# Fungsi untuk ekstraksi data dari JSON URL
def extract_data_json(json_url):
    try:
        response = requests.get(json_url)
        data = response.json()
        df = pd.DataFrame(data)
        print("Data berhasil diekstraksi.")
        return df
    except Exception as e:
        print("Gagal mengekstraksi data:", e)
        return pd.DataFrame()

# Fungsi untuk transformasi data
def transform_data(df):
    symbols = ['GOOGL', 'AMZN', 'MSFT', 'AAPL']
    filtered_df = df[df['stock_symbol'].isin(symbols)]
    high_price_df = filtered_df[filtered_df['trade_price'] > 500]
    print("Data berhasil ditransformasi.")
    return high_price_df

# Fungsi untuk load ke Google Storage
def load_data_to_firebase(df, storage_bucket, filename):
    try:
        if not firebase_admin._apps:
            cred = credentials.Certificate("keyErista.json")
            firebase_admin.initialize_app(cred, {"storageBucket": storage_bucket})
        
        df.to_parquet(filename)
        blob = storage.bucket(storage_bucket).blob(filename)
        blob.upload_from_filename(filename, content_type='parquet')
        
        print("Data berhasil diunggah ke Firebase Storage.")
    except Exception as e:
        print("Terjadi kesalahan:", e)

# Main function
if __name__ == "__main__":
    try:
        json_url = "https://gist.githubusercontent.com/nadirbslmh/93b62fdcfa694d4ec07bed9b3c94e401/raw/c07971341361e23fd4f3a880437c4d8e4ebcfafc/stock_trades.json"
        
        # Ekstraksi data
        extracted_df = extract_data_json(json_url)
        
        # Transformasi data
        if not extracted_df.empty:
            transformed_df = transform_data(extracted_df)
            
            storage_bucket = "erista-cloud.appspot.com"
            filename = "prioritas2.parquet"
            
            # Load data yang telah ditransformasi ke Google Storage
            load_data_to_firebase(transformed_df, storage_bucket, filename)
    except Exception as e:
        print("Terjadi kesalahan:", e)

Data berhasil diekstraksi.
Data berhasil ditransformasi.
Data berhasil diunggah ke Firebase Storage.
